## Extinction library creation example

This example shows how to use the extinction plugin CrysExtn [1] together with the ncmat2endf script to generate an ENDF library. Extinction is included as a modification to the coherent elastic cross-section and is included in the NCrystal plugin.

The coherent scattering cross-section is given by,
\begin{equation}
\sigma_{\textrm{coh}}^{\textrm{el}}(\lambda)=\dfrac{\lambda^{2}}{2V}\sum_{hkl}^{\lambda\leq 2d_{hkl}}d_{hkl}|F_{hkl}|^{2}E_{hkl}(\lambda,d_{hkl}),
\end{equation}
where $E_{hkl}(\lambda,d_{hkl})$ is the extinction factor. Several different models are available in the plugin, however this example will focus on primary extinction using the Sabine model [2,3]. More information on available models can be found in [1] in the file NCPhysicsModel.cc.

[1] https://github.com/XuShuqi7/ncplugin-CrysExtn

[2] Sabine, T. (1985). Aust. J. Phys. 38(3), 507-518

[3] Sabine, T. (2006). International Tables for Crystallography (2006). Vol. C. ch. 6.4, pp. 609-616

## Preamble
Install NCrystal with pip and NJOY2016 from conda-forge using mamba. Feel free to edit as you wish.

In [ ]:
always_do_pip_installs = False
try:
    import google.colab as google_colab
except ModuleNotFoundError:
    google_colab=None#not on google colab
if always_do_pip_installs or google_colab:
    %pip install -q ncrystal[all] ipympl pandas pickleshare
if google_colab:
    print("WARNING: Installing mambaforge. This will cause your notebook "
          "to say it crashed and restart itself. This is expected behaviour!!")
    %pip -q install condacolab
    import condacolab
    condacolab.install_miniforge()
try:
    import google.colab as google_colab
except ModuleNotFoundError:
    google_colab=None#not on google colab

import pathlib
import os
import shutil
import subprocess
if not shutil.which('njoy') or not shutil.which('openmc'):
    if not google_colab:
        raise RuntimeError('required dependencies not installed!')
    else:
        !mamba install njoy2016 openmc
#enable inline and interactive matplotlib plots and import NCrystal:
if google_colab:
    google_colab.output.enable_custom_widget_manager()
import matplotlib
matplotlib.rcParams.update({"figure.autolayout": True})
#always import NCrystal:
import NCrystal as NC
assert NC.version_num >=  4002000, "too old NCrystal found"
NC.test() #< quick unit test that installation works

NCrystal utilities

In [ ]:
try:
    import ncrystal_notebook_utilities
except ImportError:
    ncrystal_notebook_utilities = None
if ncrystal_notebook_utilities is None:
    %pip install ncrystal-notebook-utilities
    import ncrystal_notebook_utilities


## Install the plugin
Get the plugin CrysExtn and other dependencies. If we didn't know already how to do that, we could have found the plugin on https://github.com/mctools/ncrystal/wiki/CuratedPlugins.



In [ ]:
%pip install "git+https://github.com/XuShuqi7/ncplugin-CrysExtn"

Check that the plugin is indeed recognised by NCrystal:

In [ ]:
!nctool --plugins

We can now finally import NCrystal. And also note that we can check the plugin is available via the Python API:

In [ ]:
#always import NCrystal:
import NCrystal as NC
assert NC.version_num >=  4000000, "too old NCrystal found"
NC.test() #< quick unit test that installation works
NC.browsePlugins(dump=True) # dump=True means we print the list

# Using the plugin
Let's create a file and set the parameters of the model.

This is done by taking an existing file and adding the relevant section for the new physics through using "c.set_custom_section_data()".
The name of the pluggin is `CRYSEXTN` followed by the data used by the model.
The `CRYSEXTN` plugin currently supports both the uncorrelated and correlated model of Sabine, however with only primary extinction active, both models are the same. This example will use the correlated model. Primary extinction is controlled by the crystallite size parameter $l$, while secondary extinction is controlled by the grain size parameter $L$ and the mosacity parameter $g$ [3]. The parameters $g$ and $L$ are set to zero for only primary extinction. $l$ and $L$ have units of Ångströms and $g$ has units of 1/rad.

The data is added to the section called `@CUSTOM_CRYSEXTN`. Sabine_corr indicates the Sabine correlated model. The first parameter is $l$, the second is $g$ and the third is $L$.

In [ ]:
import pathlib
import numpy as np
import pandas
import matplotlib.pyplot as plt

c = NC.NCMATComposer('Be_sg194.ncmat')
# Set hkl, R and fraction
c.set_custom_section_data("CRYSEXTN", " Sabine_corr 5e+4  0  0 ")
c.write('Be_sg194-CrysExtn.ncmat')
print( pathlib.Path('Be_sg194-CrysExtn.ncmat').read_text())

Compare against the ideal polycrystalline Be cross-section.

In [ ]:
wl = np.linspace(0.2, 6, 10000)

cfg1 = 'Be_sg194-CrysExtn.ncmat;temp=293.6K'
pc = NC.createScatter( cfg1)
pc_a=NC.createAbsorption(cfg1)
xs1 = pc.xsect(wl=wl)

cfg2 = 'Be_sg194.ncmat;temp=293.6K'
pc = NC.createScatter( cfg2)
pc_a=NC.createAbsorption(cfg2)
xs2 = pc.xsect(wl=wl)

plt.figure()
plt.plot(NC.wl2ekin(wl), xs1, 'r-', label='293.6 K with extinction')
plt.plot(NC.wl2ekin(wl), xs2, 'b-', label='293.6 K')
plt.legend()
plt.xscale('log')
plt.xlabel('Energy [eV]')
plt.ylabel('Micro scattering cross section [b/atom]')
plt.show()

Now we setup up the parameters for the ncmat2endf script. Import the script and modify the list of allowed physics to include extinction. This is done by adding CrysExtnModel to the allowed scattering processes for the ncmat2endf script.

In [ ]:
from NCrystal import ncmat2endf
from NCrystal import _ncmat2endf_impl
_ncmat2endf_impl.allowed_scat_proc_names.add('CrysExtnModel')

Fill out the relevant metadata for the library.

In [ ]:
m = ncmat2endf.EndfMetaData()
m.set_value("alab","MyLabName")
m.set_value("libname","NuclearDataLibraryName")
m.set_value("auth","AuthorNames")
m.set_value("nlib",2)
m.set_value("matnum",{"Be":26})

Now generate the library.

In [ ]:
res = ncmat2endf.ncmat2endf(cfg1,endf_metadata=m, smin=5e-98)

Use the endf2ace utility to create an ACE file.

In [ ]:
from ncrystal_notebook_utilities.endf2ace import convert_endf_tsl_to_ace

endf_fn = 'tsl_Be.endf'
ace_fn = 'c_Be-293.6K.ace'
ace_fn, xsdir_fn = convert_endf_tsl_to_ace(endf_fn, ace_filename=ace_fn,ace_name='c_Be')

Use OpenMC to plot the data and compare against the NCrystal calculated cross-section.

In [ ]:
import openmc
thermal_data = openmc.data.ThermalScattering.from_ace('c_Be-293.6K.ace')

energy = np.logspace(np.log10(1e-3), np.log10(1.0),100000)

elastic_xs = thermal_data.elastic.xs['294K']
inelastic_xs = thermal_data.inelastic.xs['294K']

plt.figure()
total_xs = elastic_xs(energy) + inelastic_xs(energy)
plt.plot(energy, total_xs, 'b-', label='293.6 K ncmat2endf with extinction')
plt.plot(NC.wl2ekin(wl), xs1, 'r-', label='293.6 K NCrystal with extinction')
plt.legend()
plt.xscale('log')
plt.xlabel('Energy [eV]')
plt.ylabel('Micro scattering cross section [b/atom]')
plt.show()


It is seen that the data from the ENDF file cannot reproduce the exact behavior between Bragg edges.